# Подбор гиперпараметров LightGBM

## Загрузка модулей

В качестве библиотеки для подбора гиперпараметров мы будем использовать optuna, модель - LightGBM (для совместимости с пайплайном используется обёртка LGBMClassifier), отбор гиперпараметров будет производиться с помощью двух функций оптимизации.

In [2]:
# Модель
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, roc_auc_score

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from category_encoders import BinaryEncoder
from sklearn.preprocessing import RobustScaler

%matplotlib inline

In [3]:
# Предобработчик
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных. 

    Класс предобработчика данных, совмещающий бинарную кодировку 
    категориальных признаков и масштабирование числовых признаков.
    Наследуется от BaseEstimator и TransformerMixin из модуля
    base библиотеки scikit-learn для совместимости с пайплайнами
    (Pipeline из модуля sklearn.pipeline)
    
    Параметры:
        cat_features: Список категориальных признаков, которые 
            будут обработаны с помощью бинарного кодировщика.
        scale_features: Список числовых признаков, которые 
            будут обработаны с помощью Robust Scaler.
        drop_features: Список признаков, которые будут откинуты и
            не участвуют в процессах обучения и предсказания.
        rename_cols: Новые именования признаков. Требуется, если
            в признаках содержатся символы, не поддерживаемые
            моделью. 
            
    Функции:
        fit: Обучает предобработчики для дальнейшего использования.
        transform: Трансформирует датасет для дальнейшего использования. """
    
    def __init__(self, cat_features: list, scale_features: list,
                 drop_features: list, rename_cols: None | list) -> None:
        # Инициализация атрибутов
        self.cat_features = cat_features
        self.rename_cols = rename_cols
        self.drop_features = drop_features
        self.scale_features = scale_features

        # Инициализация предобработчиков
        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.robust_scaler = RobustScaler()

    def fit(self, X: pd.DataFrame, y= pd.DataFrame | None) -> object:
        """ Обучение предобработчика.

        Обучает предобработчики, на основе датасета и дополнительных признаков,
        которые выводятся из уже имеющихся (экстракция признаков).
        
        Параметры: 
            X: Экземпляр pandas.DataFrame, содержащий независимые переменные.
            y: Экземпляр pandas.DataFrame, содержащий зависимые переменные. 

        Пример:
            data_preprocessor = DataPreprocessor(cat_features, scale_features, 
                                                 drop_features, rename_cols)
            data_preprocessor.fit(X_train, y_train)
        
        Возвращает обученный предобработчик."""
        
        # Создаём копию датасета, чтобы не изменять исходный
        X_ = X.copy()
        X_.columns = self.rename_cols
        
        X_['Weekday'] += 1

        # Экстракция признаков
        X_['Provider Purchaser'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, \
                                                               X_['Purchasing Organization'].values)]
        X_['Provider Delivery option'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, \
                                                                     X_['Delivery Option'].values)]
        X_['Sum Fold'] = X_['Sum'].apply(lambda x: int(x) % 10)
        X_['ETC Difference'] = X_['Duration'] - X_['ETC Delivery']
        X_['Change Difference'] = X_['Delivery Date'] - X_['Change on Paper']
        X_['ETC Power'] = X_['ETC Difference'] ^ 2
        
        # Добавляем тригонометрические значения временных признаков
        X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Нормализация числовых признаков
        self.robust_scaler.fit(X_[self.scale_features])

        # Кодировка категориальных признаков
        X_ = self.bin_encoder.fit_transform(X_)

        # Дроп неиспользуемых признаков
        X_ = X_.drop(self.drop_features, axis=1)
        
        return self
    
    def transform(self, X) -> pd.DataFrame:
        """ Трансформирование датасета.
        
        Трансформирует датасет для дальнешего использования моделью.
        Требует предварительного обучения с помощью метода fit.
        
        Параметры: 
            X: Экземпляр pandas.DataFrame, содержащий независимые переменные. 
            
        Возвращает трансформированный датасет (экземпляр pandas.DataFrame), 
        готовый для использования моделью.
        
        Пример:
            data_preprocessor = DataPreprocessor(cat_features, scale_features, 
                                                 drop_features, rename_cols)
            data_preprocessor.fit(X_train, y_train)
            X_preprocessed = data_preprocessor.transform(X_test)"""

        # Создаём копию датасета, чтобы не изменять исходный
        X_ = X.copy()
        X_.columns = self.rename_cols

        X_['Weekday'] += 1

        # Экстракция фич
        X_['Provider Purchaser'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Purchasing Organization'].values)]
        X_['Provider Delivery option'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Delivery Option'].values)]
        X_['Sum Fold'] = X_['Sum'].apply(lambda x: int(x) % 10)
        X_['ETC Difference'] = X_['Duration'] - X_['ETC Delivery']
        X_['Change Difference'] = X_['Delivery Date'] - X_['Change on Paper']
        X_['ETC Power'] = X_['ETC Difference'] ^ 2

        # Временные фичи
        X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Нормализация
        X_[self.scale_features] = self.robust_scaler.transform(X_[self.scale_features])

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        X_ = X_.drop(self.drop_features, axis=1)

        return X_

In [ ]:
def objective_f1_macro(trial: opt.Trial) -> float:
    """ Функция оптимизации F1 (macro).
    
    Использует библиотеку optuna для подбора параметров из
    заданного диапазона. В качестве оптимизируемой метрики 
    выступает F1 (macro).
    
    Параметры:
        trial: экзмепляр optuna.Trial, представляющей собой
        историю оптимизации целевой функции.
        
    Пример:
        study = optuna.create_study(direction='maximize')
        study.optimize(objective, n_trials=100)
        
    Возвращает F1 (macro) метрику для подобранных параметров."""
    
    # Параметры
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1)
    n_estimators = trial.suggest_int('n_estimators', 1300, 3000)
    max_depth = trial.suggest_int('max_depth', 6, 21)
    max_bin = trial.suggest_int('max_bin', 64, 200),
    num_leaves = trial.suggest_int('num_leaves', 32, 260)
    reg_lambda = trial.suggest_float('l2_reg', 0.01, 1)

    # Модель
    data_preprocessor = DataPreprocessor(cat_features, scale_features, 
                                         drop_features, rename_cols)
    model = LGBMClassifier(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        num_leaves=num_leaves,
        reg_lambda=reg_lambda,
        max_bin=max_bin,
        force_col_wise=True
    )

    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])
    
    cv_score = cross_val_score(pipeline, X_general, y_general, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
    accuracy = cv_score.mean()

    return accuracy